In [22]:
'''
Author: Steve Taylor
Email: smt-at-njit-dot-edu

This is an interactive visualization that allows one to monitor daily COVID-19 patient 
confirmed case, death, and recovery counts in the United States. It is sourced from US 
CDC data maintained at: 

https://github.com/CSSEGISandData/COVID-19

which will update automatically to the latest available data upon running this IPython notebook

Set datatype to 'Confirmed Cases', 'Deaths', or 'Recovered' depending on the type of 
information you are interested in visualizing.

'''
import git
import numpy as np
import os 
import pandas as pd
import plotly_express as px
import shutil

# Inputs 
clone_flag = False # set to false to prevent recloning of data source repo
datatype = 'Confirmed Cases'  # options: 'Confirmed Cases', 'Deaths', 'Recovered'

statesdf = pd.read_csv('states.csv')    # table of state names and abbreviations
state_list = statesdf['State'].values

if clone_flag:  # clone data source repo 
    shutil.rmtree("COVID-19")
    giturl = "https://github.com/CSSEGISandData/COVID-19"
    git.Git(".").clone(giturl)

folder = 'COVID-19/csse_covid_19_data/csse_covid_19_time_series/'
deaths_flename = 'time_series_19-covid-Deaths.csv'
confirmed_flename = 'time_series_19-covid-Confirmed.csv'
recovered_flename = 'time_series_19-covid-Recovered.csv'

if datatype == 'Confirmed Cases':  # datatype switch
    df = pd.read_csv(folder+confirmed_flename)
elif datatype == 'Deaths':
    df = pd.read_csv(folder+deaths_flename)
elif datatype == 'Recovered':
    df = pd.read_csv(folder+recovered_flename)

# data wrangling     
df = df[df['Country/Region'] == 'US']
df = df[df['Province/State'].isin(state_list)]  
df = df.set_index('Province/State')
dropcols = ['Country/Region','Lat','Long']
df = df.drop(dropcols,axis=1).T
df.index = pd.to_datetime(df.index)
start_date = '2020-03-10' # no data prior to this date
df = df[start_date:]
df = df.stack().reset_index()
df.columns = ['date','state','cases']
df['state'] = df['state'].map(dict(zip(statesdf['State'],statesdf['Abbreviation'])))
df['date'] = df['date'].map(str)
colname = 'log10-'+datatype
df[colname] = np.log10(df['cases']+1) # add one to avoid np.infty errors
df['date'] = df['date'].apply(lambda x:x.split(" ")[0])

# interactive graphic
title = "COVID-19 " + datatype
px.choropleth(df, locationmode='USA-states',locations="state", color=colname,hover_name="state",hover_data=["cases","date"],animation_frame="date",scope='usa', 
             color_continuous_scale=px.colors.sequential.solar,range_color=[0,np.log10(max(df['cases']))],title=title,height=700)
